# Homework and bake-off: Sentiment analysis

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2022"

## Contents

1. [Overview](#Overview)
1. [Methodological note](#Methodological-note)
1. [Set-up](#Set-up)
1. [Train set](#Train-set)
1. [Dev sets](#Dev-sets)
1. [A softmax baseline](#A-softmax-baseline)
1. [RNNClassifier wrapper](#RNNClassifier-wrapper)
1. [Error analysis](#Error-analysis)
1. [Homework questions](#Homework-questions)
    1. [Token-level differences [1 point]](#Token-level-differences-[1-point])
    1. [Training on some of the bakeoff data [1 point]](#Training-on-some-of-the-bakeoff-data-[1-point])
    1. [A more powerful vector-averaging baseline [2 points]](#A-more-powerful-vector-averaging-baseline-[2-points])
    1. [BERT encoding [2 points]](#BERT-encoding-[2-points])
    1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])
1. [Submission instructions](#Submission-instructions)

## Overview

This homework and associated bakeoff are devoted to supervised sentiment analysis using the ternary (positive/negative/neutral) version of the Stanford Sentiment Treebank (SST-3) as well as a new dev/test dataset drawn from restaurant reviews. Our goal in introducing the new dataset is to push you to create a system that performs well in both the movie and restaurant domains.

The homework questions ask you to implement some baseline system, and the bakeoff challenge is to define a system that does well at both the SST-3 test set and the new restaurant test set. Both are ternary tasks, and our central bakeoff score is the mean of the macro-FI scores for the two datasets. This assigns equal weight to all classes and datasets regardless of size.

The SST-3 test set will be used for the bakeoff evaluation. This dataset is already publicly distributed, so we are counting on people not to cheat by developing their models on the test set. You must do all your development without using the test set at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. __Much of the scientific integrity of our field depends on people adhering to this honor code__. 

One of our goals for this homework and bakeoff is to encourage you to engage in __the basic development cycle for supervised models__, in which you

1. Design a new system. We recommend starting with something simple.
1. Use `sst.experiment` to evaluate your system, using random train/test splits initially.
1. If you have time, compare your system with others using `sst.compare_models` or `utils.mcnemar`. (For discussion, see [this notebook section](sst_02_hand_built_features.ipynb#Statistical-comparison-of-classifier-models).)
1. Return to step 1, or stop the cycle and conduct a more rigorous evaluation with hyperparameter tuning and assessment on the `dev` set.

[Error analysis](#Error-analysis) is one of the most important methods for steadily improving a system, as it facilitates a kind of human-powered hill-climbing on your ultimate objective. Often, it takes a careful human analyst just a few examples to spot a major pattern that can lead to a beneficial change to the feature representations.

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). The only constraint we need to place on your system is that it must have a `predict_one` method that can map directly from an example text to a prediction, and it must be able to make predictions without having any information beyond the text. (For example, it can't depend on knowing which task the text comes from.) See [the bakeoff section below](#Bakeoff-[1-point]) for examples of functions that conform to this specification.

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [59]:
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn

from torch_rnn_classifier import TorchRNNClassifier
from torch_tree_nn import TorchTreeNN
import sst
import utils
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [3]:
SST_HOME = os.path.join('data', 'sentiment')

In [4]:
detokenizer = TreebankWordDetokenizer()

def detokenize(s):
    return detokenizer.detokenize(s.split())

## Train set

Our primary train set is the SST-3 train set:

In [5]:
sst_train = sst.train_reader(SST_HOME)

In [6]:
sst_train.shape[0]

8544

This is the train set we will use for all the regular homework questions. You are welcome to bring in new datasets for your original system. You are also free to add `include_subtrees=True`. This is very likely to lead to better systems, but it substantially increases the overall size of the dataset (from 8,544 examples to 159,274), which will in turn substantially increase the time it takes to run experiments.

See [this notebook](sst_01_overview.ipynb) for additional details of this dataset.

## Dev sets

We have two development set. SST3-dev consists of sentences from movie reviews, just like SST-3 train:

In [7]:
sst_dev = sst.dev_reader(SST_HOME)

Our new bakeoff dev set consists of sentences from restaurant reviews:

In [8]:
bakeoff_dev = sst.bakeoff_dev_reader(SST_HOME)

In [9]:
bakeoff_dev.sample(3).to_dict(orient='records')

[{'example_id': 1598,
  'sentence': 'Our menu was fixed as part of the gift.',
  'label': 'neutral',
  'is_subtree': 0},
 {'example_id': 1401,
  'sentence': "I don't want to sit close to the door again....at least not during winter months.",
  'label': 'negative',
  'is_subtree': 0},
 {'example_id': 2089,
  'sentence': 'Upscale without the downtown drama.',
  'label': 'positive',
  'is_subtree': 0}]

Here is the label distribution:

In [10]:
bakeoff_dev.label.value_counts()

label
neutral     1019
positive     777
negative     565
Name: count, dtype: int64

The label distribution for the corresponding test set is similar to this.

## A softmax baseline

This example is here mainly as a reminder of how to use our experimental framework with linear models:

In [11]:
def unigrams_phi(text):
    return Counter(text.split())

Thin wrapper around `LogisticRegression` for the sake of `sst.experiment`:

In [12]:
def fit_softmax_classifier(X, y):
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')
    mod.fit(X, y)
    return mod

The experimental run with some notes:

In [13]:
softmax_experiment = sst.experiment(
    sst.train_reader(SST_HOME),   # Train on any data you like except SST-3 test!
    unigrams_phi,                 # Free to write your own!
    fit_softmax_classifier,       # Free to write your own!
    assess_dataframes=[sst_dev, bakeoff_dev]) # Free to change this during development!

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.272     0.690     0.390       565
     neutral      0.429     0.113     0.179      1019
    positive      0.409     0.346     0.375       777

    accuracy                          0.328      2361
   macro avg      0.370     0.383     0.315      2361
weighted avg      0.385     0.328     0.294      2361

Mean of macro-F1 scores: 0.416


`softmax_experiment` contains a lot of information that you can use for error analysis; see [this section below](#Error-analysis) for starter code.

## RNNClassifier wrapper

This section illustrates how to use `sst.experiment` with `TorchRNNClassifier`.

To featurize examples for an RNN, we can just get the words in order, letting the model take care of mapping them into an embedding space.

In [14]:
def rnn_phi(text):
    return text.split()

The model wrapper gets the vocabulary using `sst.get_vocab`. If you want to use pretrained word representations in here, then you can have `fit_rnn_classifier` build that space too; see [this notebook section for details](sst_03_neural_networks.ipynb#Pretrained-embeddings). See also [torch_model_base.py](torch_model_base.py) for details on the many optimization parameters that `TorchRNNClassifier` accepts.

In [15]:
def fit_rnn_classifier(X, y):
    sst_glove_vocab = utils.get_vocab(X, mincount=2)
    mod = TorchRNNClassifier(
        sst_glove_vocab,
        early_stopping=True)
    mod.fit(X, y)
    return mod

In [16]:
rnn_experiment = sst.experiment(
    sst.train_reader(SST_HOME),
    rnn_phi,
    fit_rnn_classifier,
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_dataframes=[sst_dev, bakeoff_dev])

Stopping after epoch 13. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 8.306065320968628

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.377     0.383     0.380       428
     neutral      0.000     0.000     0.000       229
    positive      0.395     0.592     0.474       444

    accuracy                          0.388      1101
   macro avg      0.257     0.325     0.285      1101
weighted avg      0.306     0.388     0.339      1101



/Users/zanqiliang/.pyenv/versions/3.10.4/envs/cs224u/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zanqiliang/.pyenv/versions/3.10.4/envs/cs224u/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zanqiliang/.pyenv/versions/3.10.4/envs/cs224u/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.239     0.996     0.385       565
     neutral      0.000     0.000     0.000      1019
    positive      0.200     0.001     0.003       777

    accuracy                          0.239      2361
   macro avg      0.146     0.333     0.129      2361
weighted avg      0.123     0.239     0.093      2361

Mean of macro-F1 scores: 0.207


/Users/zanqiliang/.pyenv/versions/3.10.4/envs/cs224u/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zanqiliang/.pyenv/versions/3.10.4/envs/cs224u/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zanqiliang/.pyenv/versions/3.10.4/envs/cs224u/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

## Error analysis

This section begins to build an error-analysis framework using the dicts returned by `sst.experiment`. These have the following structure:

```
'model': trained model
'phi': the feature function used
'train_dataset':
   'X': feature matrix
   'y': list of labels
   'vectorizer': DictVectorizer,
   'raw_examples': list of raw inputs, before featurizing   
'assess_datasets': list of datasets, each with the same structure as the value of 'train_dataset'
'predictions': list of lists of predictions on the assessment datasets
'metric': `score_func.__name__`, where `score_func` is an `sst.experiment` argument
'score': the `score_func` score on the each of the assessment dataasets
```
The following function just finds mistakes, and returns a `pd.DataFrame` for easy subsequent processing:

In [17]:
def find_errors(experiment):
    """Find mistaken predictions.

    Parameters
    ----------
    experiment : dict
        As returned by `sst.experiment`.

    Returns
    -------
    pd.DataFrame

    """
    dfs = []
    for i, dataset in enumerate(experiment['assess_datasets']):
        df = pd.DataFrame({
            'raw_examples': dataset['raw_examples'],
            'predicted': experiment['predictions'][i],
            'gold': dataset['y']})
        df['correct'] = df['predicted'] == df['gold']
        df['dataset'] = i
        dfs.append(df)
    return pd.concat(dfs)

In [18]:
softmax_analysis = find_errors(softmax_experiment)

In [19]:
rnn_analysis = find_errors(rnn_experiment)

Here we merge the softmax and RNN experiments into a single DataFrame:

In [20]:
analysis = softmax_analysis.merge(
    rnn_analysis, left_on='raw_examples', right_on='raw_examples')

analysis = analysis.drop('gold_y', axis=1).rename(columns={'gold_x': 'gold'})

In [21]:
analysis

,raw_examples,predicted_x,gold,correct_x,dataset_x,predicted_y,correct_y,dataset_y
0,It 's a lovely film with lovely performances b...,positive,positive,True,0,negative,False,0
1,"No one goes unindicted here , which is probabl...",negative,neutral,False,0,negative,False,0
2,And if you 're not nearly moved to tears by a ...,positive,positive,True,0,positive,True,0
3,"A warm , funny , engaging film .",positive,positive,True,0,negative,False,0
4,Uses sharp humor and insight into human nature...,positive,positive,True,0,positive,True,0
...,...,...,...,...,...,...,...,...
3459,They had added extra tables and it was a bit c...,negative,negative,True,1,negative,True,1
3460,The casino was difficult to navigate.,negative,negative,True,1,negative,True,1
3461,They turned the volume down later.,negative,neutral,False,1,negative,False,1
3462,We find it still best to explore this wonderfu...,positive,positive,True,1,negative,False,1


The following code collects a specific subset of examples; small modifications to its structure will give you different interesting subsets:

In [22]:
# Examples where the softmax model is correct, the RNN is not,
# and the gold label is 'positive'

error_group = analysis[
    (analysis['predicted_x'] == analysis['gold'])
    &
    (analysis['predicted_y'] != analysis['gold'])
    &
    (analysis['gold'] == 'positive')
]

In [23]:
error_group.shape[0]

399

In [24]:
for ex in error_group['raw_examples'].sample(5, random_state=1):
    print("="*70)
    print(ex)

My water glass never got below the halfway mark before being refilled and my wine glass stayed topped off.
The quality of the food is top shelf.
Mama's Fish House is our favorite restaurnat on Maui...perhaps anywhere.
Could have lingered if we wanted to.
Very quiet and not rushes.


## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Token-level differences [1 point]

We can begin to get a sense for how our two dev sets differ by considering the most frequent tokens from each. This question asks you to begin such analysis.

Your task: write a function `get_token_counts` that, given a `pd.DataFrame` in the format of our datasets, tokenizes the example sentences based on whitespace and creates a count distribution over all of the tokens. The function should return a `pd.Series` sorted by frequency; if you create a count dictionary `d`, then `pd.Series(d).sort_values(ascending=False)` will give you what you need.

In [25]:
def get_token_counts(df):
    pass
    ##### YOUR CODE HERE
    return df.sentence.str.split(expand=True).stack().value_counts()
    # d = Counter(df.sentence.str.split(expand=True).stack())
    # return pd.Series(d).sort_values(ascending=False)



In [26]:
def test_get_token_counts(func):
    df = pd.DataFrame([
        {'sentence': 'a a b'},
        {'sentence': 'a b a'},
        {'sentence': 'a a a b.'}])
    result = func(df)
    for token, expected in (('a', 7), ('b', 2), ('b.', 1)):
        actual = result.loc[token]
        assert actual == expected, \
            "For token {}, expected {}; got {}".format(
            token, expected, actual)

In [27]:
df = pd.DataFrame([
        {'sentence': 'a a b'},
        {'sentence': 'a b a'},
        {'sentence': 'a a a b.'}])
print(df.sentence.str.split(expand=True))
print()
print(df.sentence.str.split(expand=True).stack()) # stack cells in a single column
print()
print(df.sentence.str.split(expand=True).stack().value_counts()) # unique rows and their counts

# Counter(df.sentence.str.split())
Counter(df.sentence.str.split(expand=True).stack())


   0  1  2     3
0  a  a  b  None
1  a  b  a  None
2  a  a  a    b.

0  0     a
   1     a
   2     b
1  0     a
   1     b
   2     a
2  0     a
   1     a
   2     a
   3    b.
dtype: object

a     7
b     2
b.    1
Name: count, dtype: int64


Counter({'a': 7, 'b': 2, 'b.': 1})

In [28]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_token_counts(get_token_counts)

As you develop your original system, you might review these results. The two dev sets have different vocabularies and different low-level encoding details that are sure to impact model performance, especially when one considers that the train set is like `sst_dev` in all these respects. For additional discussion, see [this notebook section](sst_01_overview.ipynb#Tokenization).

In [29]:
get_token_counts(sst_dev)[:20]

.       1037
,        935
the      810
of       585
and      576
a        558
to       406
's       350
is       322
in       245
that     235
it       233
The      157
film     147
as       146
for      135
but      135
its      134
with     131
It       116
Name: count, dtype: int64

In [30]:
get_token_counts(bakeoff_dev)[:20]

the     1402
a        886
and      818
to       750
was      731
I        553
of       463
for      412
we       354
We       344
The      336
were     334
in       306
had      297
that     290
is       274
it       264
on       238
not      224
at       222
Name: count, dtype: int64

Me: bakeoff_dev and sst_dev have different tokenization scheme. Can't simply split on white space. Use `treebank_tokenize` or `TreebankWordDetokenizer` to bring them to the same page.

### Training on some of the bakeoff data [1 point]

We have so far presented the bakeoff dev set as purely for evaluation. Since the train set consists entirely of SST-3 data, this makes the bakeoff split especially challenging. We might be able to reduce the challenging by adding some of the bakeoff dev set to the train set, keeping some of it for evaluation. The current question asks to begin explore the effects of such training.

Your task: write a function `run_mixed_training_experiment`. The function should:

1. Take as inputs (a) a model training wrapper like `fit_softmax_classifier` and (b) an integer `bakeoff_train_size` specifying the number of examples from `bakeoff_dev` that should be included in the train set.
1. Split `bakeoff_dev` so that the first `bakeoff_train_size` examples are in the train set and the rest are used for evaluation.
1. Use `sst.experiment` with the user-supplied model training wrapper, `unigrams_phi` as defined above, and a train set that consists of SST-3 train and the train portion of `bakeoff_dev` as defined in step 2. The value of `assess_dataframes` should be a list consisting of the SST-3 dev set and the evaluation portion of `bakeoff_dev` as defined in step 2.
1. Return the return value of `sst.experiment`.

The function `test_run_mixed_training_experiment` will help you iterate to the required design.

In [31]:
def run_mixed_training_experiment(
    train_func,
    phi=unigrams_phi,
    train_dfs=[sst_train, bakeoff_dev[:1000]],
    assess_dfs=[sst_dev, bakeoff_dev[1000:]],
    vectorize=False,
):
    pass
    ##### YOUR CODE HERE
    return sst.experiment(
        train_dfs,  # Train on any data you like except SST-3 test!
        phi,
        train_func,  # Free to write your own!
        assess_dataframes=assess_dfs,  # Free to change this during development!
        vectorize=vectorize,
    )

In [32]:
def test_run_mixed_training_experiment(func):
    bakeoff_train_size = 1000
    experiment = func(fit_softmax_classifier, vectorize=True)

    assess_size = len(experiment['assess_datasets'])
    assert len(experiment['assess_datasets']) == 2, \
        ("The evaluation should be done on two datasets: "
         "SST3 and part of the bakeoff dev set. "
         "You have {} datasets.".format(assess_size))

    bakeoff_test_size = bakeoff_dev.shape[0] - bakeoff_train_size
    expected_eval_examples = bakeoff_test_size + sst_dev.shape[0]
    eval_examples = sum(len(d['raw_examples']) for d in experiment['assess_datasets'])
    assert expected_eval_examples == eval_examples, \
        "Expected {} evaluation examples; got {}".format(
        expected_eval_examples, eval_examples)

In [33]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_mixed_training_experiment(run_mixed_training_experiment)

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.627     0.671     0.648       428
     neutral      0.319     0.162     0.214       229
    positive      0.638     0.757     0.692       444

    accuracy                          0.599      1101
   macro avg      0.528     0.530     0.518      1101
weighted avg      0.567     0.599     0.576      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.471     0.412     0.440       320
     neutral      0.588     0.590     0.589       612
    positive      0.503     0.548     0.525       429

    accuracy                          0.535      1361
   macro avg      0.521     0.517     0.518      1361
weighted avg      0.534     0.535     0.534      1361

Mean of macro-F1 scores: 0.518


### A more powerful vector-averaging baseline [2 points]

In [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the ternary SST-3 problem in which each example is modeled as the mean of its GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better with these representations, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. To address this question, we want to get set up to run the experiment with a shallow neural classifier. 

Your task: write and submit a model wrapper function around `TorchShallowNeuralClassifier`. This function should implement hyperparameter search according to this specification:

* Set `early_stopping=True` for all experiments.
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `nn.Tanh()` and `nn.ReLU()`.
* For all other parameters to `TorchShallowNeuralClassifier`, use the defaults.

See [this notebook section](sst_02_hand_built_features.ipynb#Hyperparameter-search) for examples. You are not required to run a full evaluation with this function using `sst.experiment`, but we assume you will want to.

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be variation in the results.) However, the primary goal of this question is to get you thinking more about this strong baseline feature representation scheme for SST-3, so we're sort of hoping you feel compelled to try out variations on your own.

In [34]:
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

def fit_shallow_neural_classifier_with_hyperparameter_search(X, y):
    pass
    ##### YOUR CODE HERE
    model = TorchShallowNeuralClassifier(early_stopping=True)
    cv = 3
    param_grid = {
        'hidden_dim': [50, 100, 200],
        'hidden_activation': [nn.Tanh(), nn.ReLU()],
        # 'eta': [0.001, 0.01, 0.05],
        # 'l2_strength': [0.0, 0.001, 0.01],
    }
    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, model, cv, param_grid)
    return bestmod


In [35]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [36]:
def vsm_phi(text, lookup, np_func=np.mean):
    """Represent `text` as a combination of the vector of its words.

    Parameters
    ----------
    text : str

    lookup : dict
        From words to vectors.

    np_func : function (default: np.mean)
        A numpy matrix operation that can be applied columnwise,
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a
        fixed length, no matter what the size of the text is.

    Returns
    -------
    np.array, dimension `X.shape[1]`

    """
    allvecs = np.array([lookup[w] for w in text.split() if w in lookup])
    if len(allvecs) == 0:
        dim = len(next(iter(lookup.values())))
        feats = np.zeros(dim)
    else:
        feats = np_func(allvecs, axis=0)
    return feats


def glove_phi(text, np_func=np.mean):
    return vsm_phi(text, glove_lookup, np_func=np_func)

In [37]:
%%time
_ = run_mixed_training_experiment(
    fit_shallow_neural_classifier_with_hyperparameter_search, glove_phi
)

Stopping after epoch 31. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 6.889146029949188

Best params: {'hidden_activation': ReLU(), 'hidden_dim': 200}
Best score: 0.557
Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.621     0.724     0.669       428
     neutral      0.435     0.131     0.201       229
    positive      0.634     0.761     0.692       444

    accuracy                          0.616      1101
   macro avg      0.563     0.539     0.521      1101
weighted avg      0.588     0.616     0.581      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.502     0.419     0.457       320
     neutral      0.584     0.672     0.625       612
    positive      0.608     0.552     0.579       429

    accuracy                          0.575      1361
   macro avg      0.564     0.548     0.553      1361
weighted avg      0.572     0.575     0.571      1361

Mean of macro-F1 scores: 0.537
CPU times: user 39.6 s, sys: 5.99 s, total: 45.6 s
Wall time: 47.3 s


Yelp vector averaging baseline

In [38]:
import vsm

VSMDATA_HOME = os.path.join('data', 'vsmdata')

yelp20 = pd.read_csv(
    os.path.join(VSMDATA_HOME, 'yelp_window20-flat.csv.gz'), index_col=0)

yelp20_ppmi = vsm.pmi(yelp20, positive=False)

yelp20_ppmi_svd = vsm.lsa(yelp20_ppmi, k=300)

yelp_lookup = dict(zip(yelp20_ppmi_svd.index, yelp20_ppmi_svd.values))

def yelp_phi(text, np_func=np.mean):
    return vsm_phi(text, yelp_lookup, np_func=np_func)

In [39]:
%%time 
_ = run_mixed_training_experiment(
    fit_shallow_neural_classifier_with_hyperparameter_search, yelp_phi
)

Stopping after epoch 34. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 6.2196398377418528

Best params: {'hidden_activation': ReLU(), 'hidden_dim': 100}
Best score: 0.500
Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.588     0.701     0.640       428
     neutral      0.429     0.105     0.168       229
    positive      0.606     0.730     0.662       444

    accuracy                          0.589      1101
   macro avg      0.541     0.512     0.490      1101
weighted avg      0.562     0.589     0.551      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.452     0.519     0.483       320
     neutral      0.614     0.569     0.590       612
    positive      0.583     0.580     0.582       429

    accuracy                          0.561      1361
   macro avg      0.550     0.556     0.552      1361
weighted avg      0.566     0.561     0.562      1361

Mean of macro-F1 scores: 0.521
CPU times: user 35.4 s, sys: 4.47 s, total: 39.9 s
Wall time: 39.1 s


Compare with VecAvg

### BERT encoding [2 points]

We might hypothesize that encoding our examples with BERT will yield improvements over the GloVe averaging method explored in the previous question, since BERT implements a much more complex and data-driven function for this kind of combination. This question asks you to begin exploring this general hypothesis.

Your task: write a function `hf_cls_phi` that uses Hugging Face functionality to encode individual examples with BERT and returns the final output representation above the [CLS] token.

You are not required to evaluate this feature function, but it is easy to do so with `sst.experiment` and `vectorize=False` (since your feature function directly encodes every example as a vector). Your code should also be a natural basis for even more powerful approaches – for example, it might be even better to pool all the output states rather than using just the first output state. Another option is [fine-tuning](finetuning.ipynb).

In [72]:
from transformers import BertModel, BertTokenizer, AutoTokenizer, ElectraModel, AutoModel

# Instantiate a Bert model and tokenizer based on `bert_weights_name`:
bert_weights_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
model = BertModel.from_pretrained(bert_weights_name)

In [41]:
import vsm

##### YOUR CODE HERE


def hf_cls_phi(text):
    # Get the ids. `vsm.hf_encode` will help; be sure to
    # set `add_special_tokens=True`.
    ##### YOUR CODE HERE
    token_ids =  vsm.hf_encode(text, tokenizer, add_special_tokens=True)


    # Get the BERT representations. `vsm.hf_represent` will help:
    ##### YOUR CODE HERE
    reps = vsm.hf_represent(token_ids, model)


    # Index into `reps` to get the representation above [CLS].
    # The shape of `reps` should be (1, n, 768), where n is the
    # number of tokens. You need the 0th element of the 2nd dim:
    ##### YOUR CODE HERE
    cls_rep = reps[0, 0, :]


    # These conversions should ensure that you can work with the
    # representations flexibly. Feel free to change the variable
    # name:
    return cls_rep.cpu().numpy()

In [42]:
def test_hf_cls_phi(func):
    rep = func("Just testing!")

    expected_shape = (768,)
    result_shape = rep.shape
    assert rep.shape == (768,), \
        "Expected shape {}; got {}".format(
        expected_shape, result_shape)

    # String conversion to avoid precision errors:
    expected_first_val = str(0.1709)
    result_first_val = "{0:.04f}".format(rep[0])

    assert expected_first_val == result_first_val, \
        ("Unexpected representation values. Expected the "
        "first value to be {}; got {}".format(
            expected_first_val, result_first_val))

In [43]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_hf_cls_phi(hf_cls_phi)

In [44]:
%%time
_ = run_mixed_training_experiment(
    fit_shallow_neural_classifier_with_hyperparameter_search, hf_cls_phi, train_dfs=[sst_train, bakeoff_dev[:1000]]
)

# Shallow NN with Bert representations:

Stopping after epoch 28. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 5.829778611660004

Best params: {'hidden_activation': Tanh(), 'hidden_dim': 50}
Best score: 0.641
Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.698     0.822     0.755       428
     neutral      0.446     0.144     0.218       229
    positive      0.723     0.851     0.782       444

    accuracy                          0.693      1101
   macro avg      0.622     0.606     0.585      1101
weighted avg      0.656     0.693     0.654      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.565     0.613     0.588       320
     neutral      0.669     0.667     0.668       612
    positive      0.681     0.641     0.660       429

    accuracy                          0.646      1361
   macro avg      0.638     0.640     0.639      1361
weighted avg      0.648     0.646     0.647      1361

Mean of macro-F1 scores: 0.612
CPU times: user 9min 23s, sys: 3min 6s, total: 12min 30s
Wall time: 8min 36s


In [ ]:
sst_train_detok = sst_train.copy()
sst_train_detok['sentence'] = sst_train_detok['sentence'].apply(detokenize)

In [46]:
sst_dev_detok = sst_dev.copy()
sst_dev_detok['sentence'] = sst_dev_detok['sentence'].apply(detokenize)

In [53]:
%%time
_ = run_mixed_training_experiment(
    fit_shallow_neural_classifier_with_hyperparameter_search, 
    hf_cls_phi, 
    train_dfs=[sst_train_detok, bakeoff_dev[:1000]], 
    assess_dfs=[sst_dev_detok, bakeoff_dev[1000:]],
)

# Shallow NN with Bert representations and sst detok:

Stopping after epoch 28. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 5.8397810459136965

Best params: {'hidden_activation': Tanh(), 'hidden_dim': 200}
Best score: 0.632
Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.708     0.799     0.751       428
     neutral      0.426     0.275     0.334       229
    positive      0.760     0.804     0.781       444

    accuracy                          0.692      1101
   macro avg      0.631     0.626     0.622      1101
weighted avg      0.670     0.692     0.676      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.589     0.516     0.550       320
     neutral      0.631     0.768     0.693       612
    positive      0.732     0.573     0.643       429

    accuracy                          0.647      1361
   macro avg      0.651     0.619     0.629      1361
weighted avg      0.653     0.647     0.644      1361

Mean of macro-F1 scores: 0.625
CPU times: user 7min 58s, sys: 2min 21s, total: 10min 20s
Wall time: 6min 42s


Note: encoding all of SST-3 train (no subtrees) takes about 11 minutes on my 2015 iMac, CPU only (32GB).

### Your original system [3 points]

Your task is to develop an original model for the SST-3 problem and our new bakeoff dataset. There are many options. If you spend more than a few hours on this homework problem, you should consider letting it grow into your final project! Here are some relatively manageable ideas that you might try:

1. We didn't systematically evaluate the `bidirectional` option to the `TorchRNNClassifier`. Similarly, that model could be tweaked to allow multiple LSTM layers (at present there is only one), and you could try adding layers to the classifier portion of the model as well.

1. We've already glimpsed the power of rich initial word representations, and later in the course we'll see that smart initialization usually leads to a performance gain in NLP, so you could perhaps achieve a winning entry with a simple model that starts in a great place.

1. Our [practical introduction to contextual word representations](finetuning.ipynb) covers pretrained representations and interfaces that are likely to boost the performance of any system.

We want to emphasize that this needs to be an __original__ system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it. See the course website for additional guidance on how original systems will be evaluated.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.  We also ask that you report the best score your system got during development (your best average of macro-F1 scores), just to help us understand how systems performed overall.

<font color='red'>Please review the descriptions in the following comment and follow the instructions.</font>

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

In [54]:
def bert_finetune_phi(text):
    return text

In [71]:
class HfBertClassifierModel(nn.Module):
    def __init__(self, n_classes, weights_name='bert-base-cased'):
        super().__init__()
        self.n_classes = n_classes
        self.weights_name = weights_name
        self.bert = AutoModel.from_pretrained(self.weights_name)
        self.bert.train()
        self.hidden_dim = self.bert.embeddings.word_embeddings.embedding_dim
        # The only new parameters -- the classifier:
        self.classifier_layer = nn.Linear(
            self.hidden_dim, self.n_classes)

    def forward(self, indices, mask):
        reps = self.bert(
            indices, attention_mask=mask)
        return self.classifier_layer(reps.pooler_output)

In [75]:
class HfBertClassifier(TorchShallowNeuralClassifier):
    def __init__(self, weights_name, *args, **kwargs):
        self.weights_name = weights_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.weights_name)
        super().__init__(*args, **kwargs)
        self.params += ['weights_name']

    def build_graph(self):
        return HfBertClassifierModel(self.n_classes_, self.weights_name)

    def build_dataset(self, X, y=None):
        data = self.tokenizer.batch_encode_plus(
            X,
            max_length=None,
            add_special_tokens=True,
            padding='longest',
            return_attention_mask=True)
        indices = torch.tensor(data['input_ids'])
        mask = torch.tensor(data['attention_mask'])
        if y is None:
            dataset = torch.utils.data.TensorDataset(indices, mask)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(indices, mask, y)
        return dataset

In [57]:
def fit_bert_classifier(X, y):
    basemod = HfBertClassifier(
        weights_name='bert-base-cased',
        batch_size=8,  # Small batches to avoid memory overload.
        gradient_accumulation_steps=4,
        # max_iter=1,  # We'll search based on 1 iteration for efficiency.
        eta=5e-5,
        hidden_dim=100,
        n_iter_no_change=5,   # Early-stopping params are for the
        early_stopping=True)  # final evaluation.
    
    basemod.fit(X, y)
    return basemod

In [60]:
%%time
_ = run_mixed_training_experiment(
    fit_bert_classifier, 
    bert_finetune_phi, 
    train_dfs=[sst_train_detok, bakeoff_dev[:1000]], 
    assess_dfs=[sst_dev_detok, bakeoff_dev[1000:]],
)

# Shallow NN with Bert representations and sst detok:

Stopping after epoch 10. Validation score did not improve by tol=1e-05 for more than 5 epochs. Final error is 10.399276527277834

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.796     0.731     0.762       428
     neutral      0.373     0.384     0.378       229
    positive      0.765     0.813     0.788       444

    accuracy                          0.692      1101
   macro avg      0.645     0.643     0.643      1101
weighted avg      0.696     0.692     0.693      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.710     0.575     0.636       320
     neutral      0.662     0.828     0.736       612
    positive      0.792     0.620     0.695       429

    accuracy                          0.703      1361
   macro avg      0.721     0.674     0.689      1361
weighted avg      0.714     0.703     0.700      1361

Mean of macro-F1 scores: 0.666
CPU times: user 15min 38s, sys: 7min 56s, total: 23min 34s
Wall time: 33min 56s


Hidden States Classifier

In [61]:
class HfBertHiddenClassifierModel(HfBertClassifierModel):
    def forward(self, indices, mask):
        reps = self.bert(indices, attention_mask=mask)
        return self.classifier_layer(
            reps.last_hidden_state[:, 0, :]
        )  # last_hidden_state is (batch_size, seq_len, hidden_dim)


class HfBertHiddenClassifier(HfBertClassifier):
    def build_graph(self):
        return HfBertHiddenClassifierModel(self.n_classes_, self.weights_name)


def fit_bert_hidden_classifier(X, y):
    basemod = HfBertHiddenClassifier(
        weights_name="bert-base-cased",
        batch_size=8,  # Small batches to avoid memory overload.
        gradient_accumulation_steps=4,
        # max_iter=1,  # We'll search based on 1 iteration for efficiency.
        eta=5e-5,
        hidden_dim=100,
        n_iter_no_change=5,  # Early-stopping params are for the
        early_stopping=True,
    )  # final evaluation.

    basemod.fit(X, y)
    return basemod

In [64]:
%%time
_ = run_mixed_training_experiment(
    fit_bert_hidden_classifier, 
    bert_finetune_phi, 
    train_dfs=[sst_train_detok, bakeoff_dev[:1000]], 
    assess_dfs=[sst_dev_detok, bakeoff_dev[1000:]],
)

# Shallow NN with Bert hidden state

Stopping after epoch 12. Validation score did not improve by tol=1e-05 for more than 5 epochs. Final error is 7.512547805037684

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.808     0.696     0.748       428
     neutral      0.387     0.463     0.421       229
    positive      0.784     0.809     0.796       444

    accuracy                          0.693      1101
   macro avg      0.659     0.656     0.655      1101
weighted avg      0.711     0.693     0.699      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.642     0.672     0.656       320
     neutral      0.739     0.742     0.741       612
    positive      0.745     0.716     0.730       429

    accuracy                          0.717      1361
   macro avg      0.709     0.710     0.709      1361
weighted avg      0.718     0.717     0.718      1361

Mean of macro-F1 scores: 0.682
CPU times: user 18min 22s, sys: 9min 30s, total: 27min 53s
Wall time: 40min 21s


Bert hidden states mean pooling

In [65]:
class HfBertMeanClassifierModel(HfBertClassifierModel):
    def forward(self, indices, mask):
        reps = self.bert(indices, attention_mask=mask)
        return self.classifier_layer(
            reps.last_hidden_state.mean(dim=1)
        )  # last_hidden_state is (batch_size, seq_len, hidden_dim)


class HfBertMeanClassifier(HfBertClassifier):
    def build_graph(self):
        return HfBertMeanClassifierModel(self.n_classes_, self.weights_name)


def fit_bert_mean_classifier(X, y):
    basemod = HfBertMeanClassifier(
        weights_name="bert-base-cased",
        batch_size=8,  # Small batches to avoid memory overload.
        gradient_accumulation_steps=4,
        # max_iter=1,  # We'll search based on 1 iteration for efficiency.
        eta=5e-5,
        hidden_dim=100,
        n_iter_no_change=5,  # Early-stopping params are for the
        early_stopping=True,
    )  # final evaluation.

    basemod.fit(X, y)
    return basemod

In [66]:
%%time
_ = run_mixed_training_experiment(
    fit_bert_mean_classifier, 
    bert_finetune_phi, 
    train_dfs=[sst_train_detok, bakeoff_dev[:1000]], 
    assess_dfs=[sst_dev_detok, bakeoff_dev[1000:]],
)

# Shallow NN with Bert mean hidden state

Stopping after epoch 14. Validation score did not improve by tol=1e-05 for more than 5 epochs. Final error is 6.2677693535979415

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.754     0.808     0.780       428
     neutral      0.419     0.271     0.329       229
    positive      0.755     0.840     0.795       444

    accuracy                          0.709      1101
   macro avg      0.643     0.640     0.635      1101
weighted avg      0.685     0.709     0.692      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.584     0.681     0.629       320
     neutral      0.725     0.673     0.698       612
    positive      0.731     0.716     0.723       429

    accuracy                          0.688      1361
   macro avg      0.680     0.690     0.684      1361
weighted avg      0.694     0.688     0.690      1361

Mean of macro-F1 scores: 0.659
CPU times: user 22min 14s, sys: 11min 19s, total: 33min 33s
Wall time: 49min 7s


Electra

In [76]:
class HfElectraClassifierModel(HfBertClassifierModel):
    def forward(self, indices, mask):
        reps = self.bert(indices, attention_mask=mask)
        return self.classifier_layer(
            reps.last_hidden_state[:, 0, :]
        )  # last_hidden_state is (batch_size, seq_len, hidden_dim)


class HfElectraClassifier(HfBertClassifier):
    def build_graph(self):
        return HfElectraClassifierModel(self.n_classes_, self.weights_name)


def fit_electra_classifier(X, y):
    basemod = HfElectraClassifier(
        weights_name="google/electra-base-discriminator",
        batch_size=8,  # Small batches to avoid memory overload.
        gradient_accumulation_steps=4,
        # max_iter=1,  # We'll search based on 1 iteration for efficiency.
        eta=5e-5,
        hidden_dim=100,
        n_iter_no_change=5,  # Early-stopping params are for the
        early_stopping=True,
    )  # final evaluation.

    basemod.fit(X, y)
    return basemod

In [77]:
%%time
_ = run_mixed_training_experiment(
    fit_electra_classifier, 
    bert_finetune_phi, 
    train_dfs=[sst_train_detok, bakeoff_dev[:1000]], 
    assess_dfs=[sst_dev_detok, bakeoff_dev[1000:]],
)

# Shallow NN with Electra

Stopping after epoch 13. Validation score did not improve by tol=1e-05 for more than 5 epochs. Final error is 13.72697336350393

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.808     0.836     0.822       428
     neutral      0.465     0.345     0.396       229
    positive      0.803     0.883     0.841       444

    accuracy                          0.753      1101
   macro avg      0.692     0.688     0.686      1101
weighted avg      0.735     0.753     0.741      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.687     0.769     0.726       320
     neutral      0.783     0.712     0.746       612
    positive      0.751     0.781     0.766       429

    accuracy                          0.747      1361
   macro avg      0.740     0.754     0.746      1361
weighted avg      0.750     0.747     0.747      1361

Mean of macro-F1 scores: 0.716
CPU times: user 18min 18s, sys: 9min 35s, total: 27min 53s
Wall time: 41min 34s


Todo:
1. ~~sst detokenize~~
2. ~~compare pooler classifier vs hidden state classifier vs attention classifier~~
3. compare bert vs electra vs distillbert
2. use all csv data

## Bakeoff [1 point]

As we said above, the bakeoff evaluation data is the official SST test-set release and a new test set derived from the same sources and labeling methods as for `bakeoff_dev`.

For this bakeoff, you'll evaluate your original system from the above homework problem on these test sets. Our metric will be the mean of the macro-F1 values, which weights both datasets equally despite their differing sizes.

The central requirement for your system is that you have define a `predict_one` method for it that maps a text (str) directly to a label prediction – one of 'positive', 'negative', 'neutral'. If you used `sst.experiment` with `vectorize=True`, then the following function (for `softmax_experiment`) will be easy to adapt – you probably just need to change the variable `softmax_experiment` to the variable for your experiment output.

In [ ]:
def predict_one_softmax(text):
    # Singleton list of feature dicts:
    feats = [softmax_experiment['phi'](text)]
    # Vectorize to get a feature matrix:
    X = softmax_experiment['train_dataset']['vectorizer'].transform(feats)
    # Standard sklearn `predict` step:
    preds = softmax_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

If you used an RNN like the one we demoed above, then featurization is a bit more straightforward:

In [ ]:
def predict_one_rnn(text):
    # List of tokenized examples:
    X = [rnn_experiment['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = rnn_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

The following function is used to create the bakeoff submission file. Its arguments are your `predict_one` function and an output filename (str).

In [ ]:
def create_bakeoff_submission(
        predict_one_func,
        output_filename='cs224u-sentiment-bakeoff-entry.csv'):

    bakeoff_test = sst.bakeoff_test_reader(SST_HOME)
    sst_test = sst.test_reader(SST_HOME)
    bakeoff_test['dataset'] = 'bakeoff'
    sst_test['dataset'] = 'sst3'
    df = pd.concat((bakeoff_test, sst_test))

    df['prediction'] = df['sentence'].apply(predict_one_func)

    df.to_csv(output_filename, index=None)

Thus, for example, the following will create a bake-off entry based on `predict_one_softmax`:

In [ ]:
# This check ensure that the following code only runs on the local environment only.
# The following call will not be run on the autograder environment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    create_bakeoff_submission(predict_one_softmax)

This creates a file `cs224u-sentiment-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in our bakeoff prediction file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.

## Submission instructions

Review and follow the [Homework and bake-off code: Formatting guide](hw_formatting_guide.ipynb).
Please do not change the filename as described below.

Submit the following files to Gradescope:

- `hw_sentiment.ipynb` (this notebook)
- `cs224u-sentiment-bakeoff-entry.csv` (bake-off output)
